In [1]:
import array
import os
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")



In [2]:
#upload the train csv file (with image label and encoded pixels)
df_train = pd.read_csv('/Users/george/Desktop/understanding_cloud_organization/train.csv')
print(df_train.shape)
df_train.head()

(22184, 2)


,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


Splitting up data to only get the label names


In [3]:
#split the image label as image's name and image's label
splitted = df_train['Image_Label'].str.split('_',expand = True)
df_train['label'] = splitted[1]
df_train['image_name'] = splitted[0]
df_train['mask'] = ~df_train['EncodedPixels'].isna()
df_train.head()


,Image_Label,EncodedPixels,label,image_name,mask
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...,Fish,0011165.jpg,True
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...,Flower,0011165.jpg,True
2,0011165.jpg_Gravel,NaN,Gravel,0011165.jpg,False
3,0011165.jpg_Sugar,NaN,Sugar,0011165.jpg,False
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...,Fish,002be4f.jpg,True


In [4]:
#replace the empty into NaN value of ENcodedPixels and label
m = len(df_train['EncodedPixels'])
df_train['EncodedPixels'].fillna("0 0", inplace = True)
df_train['EncodedPixels']

g = len(df_train['label'])
df_train['label'].fillna("0 0", inplace = True)
df_train['label']

0          Fish
1        Flower
2        Gravel
3         Sugar
4          Fish
          ...  
22179     Sugar
22180      Fish
22181    Flower
22182    Gravel
22183     Sugar
Name: label, Length: 22184, dtype: object

Began training the labels and EncodedPixels


In [125]:

features = ['label']

X = df_train[features]
y = df_train['EncodedPixels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)

Since Decison tree only gets floats and the data is in string this has to be used

In [126]:
le = LabelEncoder()
le.fit(X_train["label"].astype(str))
X_test["label"] = le.transform(X_test["label"].astype(str))
X_train["label"] = le.transform(X_train["label"].astype(str))

<ipython-input-126-d13843feaeb9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["label"] = le.transform(X_test["label"].astype(str))
<ipython-input-126-d13843feaeb9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["label"] = le.transform(X_train["label"].astype(str))


Decision Tree Classifier

In [127]:
myDecisionTree = DecisionTreeClassifier(random_state = 2)
myDecisionTree.fit(X_train, y_train)
yPerdictDt = myDecisionTree.predict(X_test)
dtScore = accuracy_score(y_test, yPerdictDt)
print ("Accuracy: " + str(dtScore))

yPerdictProbabilityDt = myDecisionTree.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, yPerdictProbabilityDt[:,1], pos_label = 1)
AUC = auc(fpr, tpr)

Accuracy: 0.46829927884615385


/Users/george/Desktop/School/Spring 2020/MachineLearning/venv2.0/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:806: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "


In [ ]:
#PartE
myAdaBoost = AdaBoostClassifier(n_estimators=29, random_state=2)
myAdaBoost.fit(X_train, y_train)
yPredictAda = myAdaBoost.predict(X_test)
adaScore = accuracy_score(y_test, yPredictAda)
print("Accuracy: " + str(adaScore))

yPerdictProbabilityAda = myAdaBoost.predict_proba(X_test)
fpr3, tpr3, thresholds3 = roc_curve(y_test, yPerdictProbabilityAda[:,1], pos_label=1)
AUC3 = auc(fpr3, tpr3)
print ('AUC: ' + str(AUC3))

In [ ]:
#PartG
myRandomForest = RandomForestClassifier(n_estimators=29, bootstrap=True, random_state=2)
myRandomForest.fit(X_train, y_train)
yPredictRf = myRandomForest.predict(X_test)
rfScore = accuracy_score(y_test, yPredictRf)
print("Accuracy: " + str(rfScore))

yPerdictProbabilityRf = myRandomForest.predict_proba(X_test)
fpr5, tpr5,thresholds5 = roc_curve(y_test, yPerdictProbabilityRf[:,1], pos_label=1)
AUC5 = auc(fpr5, tpr5)
print('AUC: ' + str(AUC5))

In [ ]:
#PartH

plt.figure()
plt.plot(fpr, tpr, color = 'green', lw = 2, label = 'Decision Tree(area = %0.2f)'%AUC)

plt.plot(fpr2, tpr2, color = 'red', lw = 2, label = 'BAgging Curve (area = %0.2f)'%AUC2)

plt.plot(fpr3, tpr3, color = 'blue', lw = 2, label = 'AdaBoost Curve (area = %0.2f)'%AUC3)

plt.plot(fpr5, tpr5, color = 'orange', lw = 2, label = 'Random Forest Curve (area = %0.2f)'%AUC5)

plt.plot([0, 1], [0, 1], color = 'brown', lw = 1, linestyle = '--')

plt.xlim([-0.005, 1.005])
plt.ylim([0.0, 1.01])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cancer Predictor')
plt.legend(loc = "lower right")
plt.show()

In [ ]:
print("Decision Tree Score: " + str(dtScore))
print ("Voting Score: " + str(majorityScore))
print ("AdaBoost Score: " + str(adaScore))
print ("Random Forest Score: " +  str(rfScore))

In [ ]:
print ("Decision Tree AUC: " + str(AUC))
print ("Voting AUC: " + str(AUC2))
print ("AdaBoost AUC: " + str(AUC3))
print ("Random Forest AUC: " + str(AUC5))

